In [1]:
import esda
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import libpysal as lps
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium
import branca
import psycopg2
import sqlalchemy
%matplotlib inline

# Avaluos manzana

In [2]:
Avaluo_cat_man = gpd.read_file('/mnt/d/Documentos/DS4A/initialData/Avaluo Catastral Manzana/AvaluoCatastralManzana/Avaluo_Manzana.shp')
Avaluo_com_man = gpd.read_file('/mnt/d/Documentos/DS4A/initialData/Avaluo Comercial Manzana/AvaluoComercialManzana/Avaluo_Manzana.shp')

In [12]:
Avaluo_cat_man.head()

,OBJECTID,MANZANA_ID,CP_TERR_AR,GRUPOP_TER,AVALUO_COM,AVALUO_CAT,OBSERVACIO,GLOBALID,SHAPE_Leng,SHAPE_Area,geometry
0,1,009259086,N,RESIDENCIAL,951874.0,733096.0,Este valor corresponde a la mediana y puede di...,{70BABE94-C17D-48FC-ADD0-BEF0283A45B4},0.002464,1.213815e-07,"POLYGON ((-74.12100 4.74751, -74.12101 4.74747..."
1,2,001355027,N,RESIDENCIAL,973208.0,666239.0,Este valor corresponde a la mediana y puede di...,{73AA8E80-CC4E-4DF5-887F-E1620A2356E4},0.000684,2.818937e-08,"POLYGON ((-74.09776 4.55222, -74.09776 4.55221..."
2,3,001355010,N,RESIDENCIAL,1100000.0,818602.0,Este valor corresponde a la mediana y puede di...,{ED6BDEF8-72FB-40F3-B4B5-A979800F5B47},0.000875,2.915943e-08,"POLYGON ((-74.09745 4.55194, -74.09747 4.55190..."
3,4,001355012,N,RESIDENCIAL,687104.0,469428.0,Este valor corresponde a la mediana y puede di...,{2EBCB819-9A4E-4F41-9A8B-0578AE28045A},0.001018,3.175301e-08,"POLYGON ((-74.09723 4.55258, -74.09718 4.55258..."
4,5,001355021,N,RESIDENCIAL,697258.0,474940.0,Este valor corresponde a la mediana y puede di...,{F25CC9EA-C4FA-49D6-9878-5EF2EBEE73FA},0.002131,9.812038e-08,"POLYGON ((-74.09668 4.55296, -74.09673 4.55288..."


In [5]:
Avaluo_cat_man == Avaluo_com_man 

,OBJECTID,MANZANA_ID,CP_TERR_AR,GRUPOP_TER,AVALUO_COM,AVALUO_CAT,OBSERVACIO,GLOBALID,SHAPE_Leng,SHAPE_Area,geometry
0,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...
43842,True,True,True,True,True,True,True,True,True,True,True
43843,True,True,True,True,True,True,True,True,True,True,True
43844,True,False,False,False,True,True,True,True,True,True,True
43845,True,True,True,True,True,True,True,True,True,True,True


In [13]:
Avaluo_cat_man.dtypes

OBJECTID         int64
MANZANA_ID      object
CP_TERR_AR      object
GRUPOP_TER      object
AVALUO_COM     float64
AVALUO_CAT     float64
OBSERVACIO      object
GLOBALID        object
SHAPE_Leng     float64
SHAPE_Area     float64
geometry      geometry
dtype: object

In [15]:
Avaluo_cat_man.isnull().any()

OBJECTID      False
MANZANA_ID     True
CP_TERR_AR     True
GRUPOP_TER     True
AVALUO_COM    False
AVALUO_CAT    False
OBSERVACIO    False
GLOBALID      False
SHAPE_Leng    False
SHAPE_Area    False
geometry      False
dtype: bool

# Ofertas

In [16]:
ofertas = pd.read_csv('/mnt/d/Documentos/DS4A/initialData/Ofertas/Ofertar.csv',sep=';',decimal=',')
ofertas.head()

,Nombre Barrio,Codigo Barrio,Codigo Manzana,Codigo Predio,Codigo Construccion,Codigo Resto,Fecha De Cargue,Direccion,Codigo Oferta,Tipo Oferta,Tipo Inmueble,Fuente,Chip,Area Terreno,Area Construida,Vr Inicial Venta,Vr Final Venta,Vr Inicial Arriendo,Vr Final Arriendo
0,ESTACION CENTRAL,6204,17,15,0,0,2019-09-30T00:00:00Z,KR 33 17 27,500000.0,VENTA,Bodega,CAMPO,AAA0251LMRJ,2060.80,2306,7.000000e+09,6.880300e+09,0.0,0.0
1,COOPERATIVA DE SUB-OFICIALES,6501,3,10,0,0,2019-09-30T00:00:00Z,KR 71F 12 02,500001.0,VENTA,Casa uso mixto,CAMPO,AAA0080RXPP,100.10,"211,3",0.000000e+00,0.000000e+00,0.0,0.0
2,ESTACION CENTRAL,6204,4,4,1,6011,2019-09-30T00:00:00Z,CL 14 30 17 AP 611,500002.0,ARRIENDO,Apartamento,CAMPO,AAA0164RCHY,18.04,"50,44",0.000000e+00,0.000000e+00,0.0,0.0
3,ARBORIZADORA ALTA,2568,10,14,0,0,2019-09-30T00:00:00Z,CL 74 SUR 38A 60,500003.0,ARRIENDO PARCIAL,Casa uso mixto,CAMPO,AAA0028RFAF,42.00,42,0.000000e+00,0.000000e+00,1200000.0,1178000.0
4,ARBORIZADORA ALTA,2568,10,12,0,0,2019-09-30T00:00:00Z,CL 74 SUR 38A 52,500004.0,VENTA,Casa uso mixto,CAMPO,AAA0028REYN,42.00,"130,2",1.500000e+08,1.453350e+08,0.0,0.0


In [17]:
ofertas.isnull().any()

Nombre Barrio          False
Codigo Barrio          False
Codigo Manzana         False
Codigo Predio          False
Codigo Construccion    False
Codigo Resto           False
Fecha De Cargue        False
Direccion              False
Codigo Oferta          False
Tipo Oferta            False
Tipo Inmueble          False
Fuente                 False
Chip                   False
Area Terreno           False
Area Construida        False
Vr Inicial Venta       False
Vr Final Venta         False
Vr Inicial Arriendo    False
Vr Final Arriendo      False
dtype: bool

In [11]:
ofertas.dtypes

Nombre Barrio           object
Codigo Barrio            int64
Codigo Manzana           int64
Codigo Predio            int64
Codigo Construccion      int64
Codigo Resto             int64
Fecha De Cargue         object
Direccion               object
Codigo Oferta          float64
Tipo Oferta             object
Tipo Inmueble           object
Fuente                  object
Chip                    object
Area Terreno           float64
Area Construida         object
Vr Inicial Venta       float64
Vr Final Venta         float64
Vr Inicial Arriendo    float64
Vr Final Arriendo      float64
dtype: object

# Valor referencia

In [13]:
Valor_referencia = gpd.read_file('/mnt/d/Documentos/DS4A/initialData/Valor Referencia del Suelo/Valor_Ref/Valor_Ref.shp')

In [14]:
Valor_referencia.head()

,MANCODIGO,V_REF,AÑO,SHAPE_AREA,SHAPE_LEN,geometry
0,009109014,1420000.0,2016-01-01,5.564496e-07,0.003106,"POLYGON ((-74.07808 4.70739, -74.07808 4.70726..."
1,009111022,2200000.0,2016-01-01,2.032584e-06,0.005819,"POLYGON ((-74.06901 4.70363, -74.06933 4.70355..."
2,009111025,2100000.0,2016-01-01,5.573527e-07,0.003223,"POLYGON ((-74.06507 4.70105, -74.06507 4.70105..."
3,009113030,800000.0,2016-01-01,8.663016e-06,0.021766,"POLYGON ((-74.06706 4.76573, -74.06760 4.76499..."
4,009115041,2200000.0,2016-01-01,1.032727e-06,0.004514,"POLYGON ((-74.07292 4.72923, -74.07292 4.72895..."


In [18]:
Valor_referencia.isnull().any()

MANCODIGO     False
V_REF         False
AÑO           False
SHAPE_AREA    False
SHAPE_LEN     False
geometry      False
dtype: bool

In [19]:
Valor_referencia.dtypes

MANCODIGO       object
V_REF          float64
AÑO             object
SHAPE_AREA     float64
SHAPE_LEN      float64
geometry      geometry
dtype: object

# Mapa de referencia

In [55]:
con = psycopg2.connect(database='MapaReferencia',user='postgres',password='152622',host='localhost')
cursor = con.cursor()

In [56]:
cursor.execute("""select schema_name
from information_schema.schemata;""")
schemas = []
for s in cursor.fetchall():
    schemas.append(s)

In [58]:
#para leer todas las tablas creo que no es necesario y se demora mucho
dictfinal = {}
for s in schemas[6:7]:
    schema = s[0]
    cursor.execute(f"""SELECT table_name FROM information_schema.tables 
    WHERE table_schema = '{schema}'""")
    temp_dict = {}
    for table in cursor.fetchall():
        sql = f'SELECT * FROM {schema}.{table[0]}'
        temp_dict[table] = gpd.GeoDataFrame.from_postgis(sql, con, geom_col='geom') 
    dictfinal[schema] = temp_dict

ValueError: Query missing geometry column 'geom'

In [36]:
sql_sectCat = 'SELECT * FROM area_catastral.scat'
sql_zhs = 'SELECT * FROM zona_hidrografica.subcuenca'

In [8]:
SectorCatastral = gpd.GeoDataFrame.from_postgis(sql_sectCat, con, geom_col='geom')

In [37]:
ZonaHidrogaficaSubcuenca = gpd.GeoDataFrame.from_postgis(sql_zhs, con, geom_col='geom')

In [10]:
SectorCatastral.head()

,gid,scacodigo,scatipo,scanombre,geom
0,1,002598,0,EL MOCHUELO II URBANO,"MULTIPOLYGON (((-74.12983 4.53681, -74.12938 4..."
1,2,004524,0,SAN PABLO BOSA,"MULTIPOLYGON (((-74.19447 4.60992, -74.19288 4..."
2,3,001344,0,LOS SOCHES,"MULTIPOLYGON (((-74.08600 4.50096, -74.08693 4..."
3,4,002630,0,EL PORTAL DEL DIVINO,"MULTIPOLYGON (((-74.09576 4.48804, -74.09586 4..."
4,7,005103,0,SAN MIGUEL,"MULTIPOLYGON (((-74.07701 4.65949, -74.07734 4..."


In [38]:
ZonaHidrogaficaSubcuenca.head()

,gid,codch,nomch,codsch,nomsch,geom
0,1,2120.0,Río Bogotá,212005.0,Embalse del Muña,"MULTIPOLYGON (((-74.24142 4.54149, -74.23972 4..."
1,2,2120.0,Río Bogotá,212002.0,Río Calandaima,"MULTIPOLYGON (((-74.49380 4.54855, -74.49311 4..."
2,3,2120.0,Río Bogotá,212001.0,Río Bajo Bogotá,"MULTIPOLYGON (((-74.61330 4.58091, -74.61265 4..."
3,4,2120.0,Río Bogotá,212006.0,Río Soacha,"MULTIPOLYGON (((-74.17585 4.50547, -74.17563 4..."
4,5,2120.0,Río Bogotá,212007.0,Río Tunjuelo,"MULTIPOLYGON (((-74.17568 4.50378, -74.17564 4..."
